In [1]:
import requests
import string
from datetime import datetime

from lxml import etree
from lxml import html

from pandas import DataFrame as df

In [15]:

queryParam = {'fromDate': '2016-1-1'}
# proxies = {'http': 'proxy.wellsfargo.com:8080', 'https': 'proxy.wellsfargo.com:8080'}
parser = etree.HTMLParser()

# r = requests.get('http://www.hongkongairport.com/flightinfo/chi/real_arrinfo.do', params=queryParam, proxies=proxies)
r = requests.get('http://www.hongkongairport.com/flightinfo/chi/real_arrinfo.do', params=queryParam)

r.url

u'http://www.hongkongairport.com/flightinfo/chi/real_arrinfo.do?fromDate=2016-1-1'

In [16]:
currentDate = datetime.now()
yyyy = currentDate.year
mm = currentDate.month
dd = currentDate.day

In [29]:
flightTable = []

tree = etree.fromstring(r.content, parser=parser)
trList =tree.xpath('//table[@id=\'list\']/tr[not(@mr)]')

for tr in trList:
    if 'date' in tr.attrib:
        dateStr = tr.attrib['date'].split(',')[1]
        yyyy=int(dateStr.split('-')[0])
        mm=int(dateStr.split('-')[1])
        dd=int(dateStr.split('-')[2])
    
    scheduledTimeStr = tr.getchildren()[0].text
    schHH = int(scheduledTimeStr.split(':')[0])
    schMM = int(scheduledTimeStr.split(':')[1])
    
    scheduledTime = datetime(yyyy, mm, dd, schHH, schMM)
    
    flightCode = tr.getchildren()[1].text
    
    city = tr.getchildren()[2].text
    
    airline = ''
    airlineRaw = tr.getchildren()[3].text
    for airlineStr in airlineRaw.split(','):
        if airlineStr.find('wm') == -1:
            airline = airline + ',' + airlineStr
    airline = airline[1:]
    
    gate = tr.getchildren()[4].text
    
    arriveTimeStr = tr.getchildren()[5].text
    if arriveTimeStr.find(':') == -1 :
        arriveTime = None
    elif arriveTimeStr.find('(') == -1 :
        timeStr = arriveTimeStr.split(' ')[1]
        arriveTime = datetime(yyyy, mm, dd, int(timeStr.split(':')[0]), int(timeStr.split(':')[1]))
    else :
        dateStr = arriveTimeStr.split('(')[1].split(')')[0]
        timeStr = arriveTimeStr.split(' ')[1]
        
        yyyyActual = int(dateStr.split('/')[2])
        mmActual = int(dateStr.split('/')[1])
        ddActual = int(dateStr.split('/')[0])
        
        hourActual = int(timeStr.split(':')[0])
        minActual = int(timeStr.split(':')[1])
        
        arriveTime = datetime(yyyyActual, mmActual, ddActual, hourActual, minActual)
        
    flightEntry = {'scheduledTime': scheduledTime, 
                  'flightCode': flightCode,
                  'city': city,
                  'airline': airline,
                  'gate': gate,
                  'arriveTime': arriveTime}
    
    flightTable.append(flightEntry)
    
flightData = df(flightTable)

flightData.head()


,airline,arriveTime,city,flightCode,gate,scheduledTime
0,"港龍航空公司,中國國際航空公司,國泰航空公司",2016-01-01 00:18:00,杭州,"KA 637,CA 6507,CX 5637",B,2015-12-31 21:40:00
1,韓亞航空,2016-01-01 00:53:00,首爾/仁川,OZ 745,B,2015-12-31 22:30:00
2,"國泰航空公司,英國航空公司,芬蘭航空,越南航空公司",2016-01-01 00:08:00,胡志明市,"CX 764,BA 4562,AY 5852,VN 3560",B,2015-12-31 23:10:00
3,"港龍航空公司,國泰航空公司",2016-01-01 00:09:00,上海/浦東,"KA 809,CX 5809",B,2015-12-31 23:25:00
4,香港快運航空,2016-01-01 00:30:00,大阪/關西,UO 689,B,2016-01-01 00:05:00


In [30]:
flightData[flightData.arriveTime.isnull()]

,airline,arriveTime,city,flightCode,gate,scheduledTime
28,香港快運航空,NaT,大阪/關西,UO 1671,,2016-01-01 05:55:00
372,"港龍航空公司,國泰航空公司",NaT,上海/浦東,"KA 875,CX 5875",,2016-01-01 19:25:00
378,廈門航空,NaT,福州,MF 8001,,2016-01-01 19:40:00
462,香港航空,NaT,遵義,HX 2207,,2016-01-01 22:35:00
497,真航空,NaT,首爾/仁川,LJ 127,,2016-01-01 23:55:00
